# LAB3 - N-Puzzle

In [43]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import itertools
from queue import PriorityQueue

### INITIALIZATION

In [44]:
PUZZLE_DIM = 3

In [45]:
class Action:
    def __init__(self, pos1, pos2):
        self.pos1 = pos1
        self.pos2 = pos2
    def __str__(self):
        return f"({self.pos1}, {self.pos2})"
class Parent:
    def __init__(self, state, action, parent):
        self.state = state
        self.action = action
        self.parent = parent

In [46]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(index[0]) for index in np.where(state == 0)]#return the position of the empty tile
    actions = list()
    if x > 0:
        actions.append(Action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(Action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(Action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(Action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: Action) -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def heuristic_simple(state: np.ndarray, goal_state:np.ndarray) -> int:
    return int(np.sum(state != goal_state))

def enhance_heuristic_manhattan(state: np.ndarray, goal_state:np.ndarray) -> int:
    #manhattan distance
    size = state.shape[0]
    total_distance = 0
    for row in range(size):
        for col in range(size):
            value = state[row, col]
            if value != 0:  # Skip the empty tile
                goal_row, goal_col = np.where(goal_state == value)
                total_distance += abs(row - goal_row[0]) + abs(col - goal_col[0])
    # 2*linear conflict because it is at least 2 moves to resolve a linear conflict
    return total_distance + 2*linear_conflict(state, goal_state)


def linear_conflict(puzzle, goal):
    size = puzzle.shape[0]
    total_conflicts = 0

    def count_conflicts(line, goal_line):
        conflicts = 0
        for i in range(len(line)):
            for j in range(i + 1, len(line)):
                # for each pair of tiles in the same row/column
                if line[i] in goal_line and line[j] in goal_line:
                    # find their goal positions
                    goal_i = np.where(goal_line == line[i])[0][0]
                    goal_j = np.where(goal_line == line[j])[0][0]
                    if goal_i > goal_j:  # they are in reverse order w.r.t goal state, so they are blocking each
                        conflicts += 1
        return conflicts

    # Check rows for linear conflicts
    for row in range(size):
        total_conflicts += count_conflicts(puzzle[row, :], goal[row, :])

    # Check columns for linear conflicts
    for col in range(size):
        total_conflicts += count_conflicts(puzzle[:, col], goal[:, col])

    return total_conflicts
def reconstruct_path(node: Parent):
    path = []
    act=None
    while node is not None:
        path.append((node.state,act))
        act=node.action
        node = node.parent
    return path[::-1]
        

In [47]:
def A_star_search(start_state: np.ndarray, goal_state: np.ndarray):
    evaluated_actions = 0

    # Initialize frontier and visited set
    visited = {}
    
    goal_tuple = tuple(map(tuple, goal_state))# Convert states to immutable tuples to use them as keys in the visited set
    counter = itertools.count() # Counter to break ties in the priority queue
    frontier = PriorityQueue()
    frontier.put((0, next(counter),start_state, 0, None, None))  # Priority, state, g, last_action, parent

    while not frontier.empty():
        _, _, current_state, g, last_action, parent = frontier.get()

        current_tuple = tuple(map(tuple, current_state))
        if current_tuple == goal_tuple:
            return reconstruct_path(Parent(current_state, last_action, parent)),evaluated_actions

        if current_tuple not in visited:
            visited[current_tuple] = g # Save the cost to reach the current state in order to avoid revisiting it in the future
            for action in available_actions(current_state):
                evaluated_actions += 1
                new_state = do_action(current_state, action)
                new_tuple = tuple(map(tuple, new_state))
                if new_tuple not in visited or visited[current_tuple] > g+1:
                    f = g + 1 + enhance_heuristic_manhattan(new_state, goal_state)
                    frontier.put((f, next(counter),new_state, g + 1, action, Parent(current_state, last_action, parent)))
    return None, None

## Initializations

In [48]:

RANDOMIZE_STEPS = 1000
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
state_init = goal_state.copy()
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state_init = do_action(state_init, choice(available_actions(state_init)))


Randomizing: 100%|██████████| 1000/1000 [00:00<00:00, 192744.08it/s]


## A* Search Algorithm

In [49]:
# count the time in milliseconds
import time
start_time = time.time()
path, cost= A_star_search(state_init, goal_state)
final_time = (time.time() - start_time)

if path is None:
    print("No solution found")
else:
    for state, action in path:
        print(f"{state}->{action}")
    print(f"quality: {len(path)}")
    print(f"cost: {cost}")
    print(f"time: {final_time}s")

[[2 3 4]
 [6 8 7]
 [0 1 5]]->((2, 0), (2, 1))
[[2 3 4]
 [6 8 7]
 [1 0 5]]->((2, 1), (1, 1))
[[2 3 4]
 [6 0 7]
 [1 8 5]]->((1, 1), (1, 0))
[[2 3 4]
 [0 6 7]
 [1 8 5]]->((1, 0), (2, 0))
[[2 3 4]
 [1 6 7]
 [0 8 5]]->((2, 0), (2, 1))
[[2 3 4]
 [1 6 7]
 [8 0 5]]->((2, 1), (1, 1))
[[2 3 4]
 [1 0 7]
 [8 6 5]]->((1, 1), (1, 2))
[[2 3 4]
 [1 7 0]
 [8 6 5]]->((1, 2), (0, 2))
[[2 3 0]
 [1 7 4]
 [8 6 5]]->((0, 2), (0, 1))
[[2 0 3]
 [1 7 4]
 [8 6 5]]->((0, 1), (0, 0))
[[0 2 3]
 [1 7 4]
 [8 6 5]]->((0, 0), (1, 0))
[[1 2 3]
 [0 7 4]
 [8 6 5]]->((1, 0), (1, 1))
[[1 2 3]
 [7 0 4]
 [8 6 5]]->((1, 1), (1, 2))
[[1 2 3]
 [7 4 0]
 [8 6 5]]->((1, 2), (2, 2))
[[1 2 3]
 [7 4 5]
 [8 6 0]]->((2, 2), (2, 1))
[[1 2 3]
 [7 4 5]
 [8 0 6]]->((2, 1), (2, 0))
[[1 2 3]
 [7 4 5]
 [0 8 6]]->((2, 0), (1, 0))
[[1 2 3]
 [0 4 5]
 [7 8 6]]->((1, 0), (1, 1))
[[1 2 3]
 [4 0 5]
 [7 8 6]]->((1, 1), (1, 2))
[[1 2 3]
 [4 5 0]
 [7 8 6]]->((1, 2), (2, 2))
[[1 2 3]
 [4 5 6]
 [7 8 0]]->None
quality: 21
cost: 509
time: 0.0448331832885742